In [1]:
%uv pip install scikit-learn

/Users/s.mallet/passau/dlnlp/.venv/bin/python: No module named uv
Note: you may need to restart the kernel to use updated packages.


# Deep Learning for Natural Language and Code: Exercise 1

In [1]:
import os
from sklearn.datasets import load_svmlight_file

# Task 1: LIBSVM - BOW

## Load data

1. Download the dataset from [here](https://ai.stanford.edu/%7Eamaas/data/sentiment)
1. Copy the dataset next to this Jupyter (.ipynb file)
1. Install:
    * Sklearn (This library is only allowed to use for reading the BOW in LIBSVM format)

In [6]:
def read_bow_libsvm(path_data):
    """ Function for reading the data in the LIBSVM format (.feat files). 
    
    Parameters
    ----------
    dirname: str
        path of this file 
    path_data: str
        path of the folder that contains the data that is going to be used. (should be test or train)
        
    Returns
    ---------
    X,y: array_like
        Data arrays, X is an array of shape [#documents of the dataset, #words in the vocabulary], y is an array of shape [#documents,] 
    """
    
    X, y = load_svmlight_file(os.path.join(".","aclImdb", path_data, "labeledBow.feat")) 
    
    # why?
    X = X.todense()
    
    return X, y

In [2]:
def read_vocab():
    """ Function for reading the vocabulary (.vocab file). 
    
    Parameters
    ----------
    None
        
    Returns
    ---------
    vocab: list
        list with the values of y ...... 
    """
    
    path_vocab = os.path.join(".", "aclImdb", "imdb.vocab")
    
    with open(path_vocab) as f:
        lines = f.read()

    lines = lines.split('\n')
    
    vocab = []
    for line in lines:
        vocab.append(line)
    
    return vocab

In [10]:
# Read the vocabulary
vocabulary = read_vocab()
# Read the the bag of words and the Y for the training data
X, y = read_bow_libsvm('train')
# Read the the bag of words and the Y for the test data
X_test, y_test = read_bow_libsvm('test')

## Understanding the LIBSVM-BOW
1. What are the values in y? 
1. Why do we use  todense() in the read_bow_libsvm function?
1. How could you know which word in the vocabulary corresponds to each index in the BOW?

In [14]:
# play with the previous loaded variables and understand the values of each one.


# Task 2: Bag of Words (BOW)
## Load Raw text and scores 

1. Be sure to have downloaded the dataset from the link provided in the exercise and have read the README file
1. Be sure to have copied the dataset next to this Jupyter (.ipynb file)
1. Be sure to have installed:
    * Numpy
    * NLTK (only for the stemming process)
    * Sklearn (only for building a Random Forest)
1. In this part of the exercise it is not allowed to use Sklearn
1. Build the Bag Of Words (BOW) with the raw data, for this you need to:
    * Tokenize on spaces and punctuation
    * Lower case
    * Remove punctuation
    * Remove terms appearing more often than X percent, this X percent should be variable. Which means that you should be able to change the percentage as a parameter.
    * Use NLTK porter stemmer
1. Build a classifier with the BOW previously built. Take into account:
    * The RF should be a binary classification positive (i.e., score >=7) and negative (i.e., score <= 4)
    * Test the classifier with the test data

In [2]:
import re
import glob
import string
import numpy as np
from nltk.stem.porter import *
from sklearn.ensemble import RandomForestClassifier
stemmer = PorterStemmer()

## Load data

Read all the training data, including the reviews and the scores associated to each one. Be sure to explore the data and learn characteristics of them, such as the type of encoding and special characters. 

## Clean HTML and tokenize text
Clean the review, handle the special characters, remove the html tags and tokenize the text based on the instructions given in the exercise sheet. 

## Convert to lower case and remove punctuation

## Remove X percentage and build vocabulary 
Remove all the tokens that do not meet the requirements based on the exercise sheet and build the vocabulary. 

## Use Porter Stemmer for stemming

## Build the bag of words (BOW)
For building the matrix for the representation of bag of words use the previously built vocabulary and tokens for each review.

# Task 3: Comparing BOWs

1. Use the previous steps to build a bag of words with the training data in which the tokens that appear more than 1% are discarded. 
1. Compare your BOW with LIBSVM BOW. 

# Task 4: Train a Random Forest and test it


# Task 5: Markov chain
Tip: For memory optimization use sparse structures not a matrix mostrly filled with zeros

## Pre-process data
Read the data and using the previous built functions for the BOW representation create a list of words per each review

## Chain words
Identify all the possible pairs of words (w0, w1) in all the reviews

## Initialize the Markov's Chain

## Generate data

Here you could also try to generate words for the unlabeled part of the dataset. Try to meassure the quality of the model